In [ ]:
import numpy as np
import xarray as xr
import xesmf as xe

from fjordssim_notebooks.regrid import replace_surrounded_and_clusters

In [ ]:
ds_in = xr.open_dataset("/home/sia/FjordsSim_data/oslofjord/Grid/OF160_grid_v2.nc")

In [ ]:
print(f"Lat min: {ds_in.lat_rho.values.min()}")
print(f"Lat max: {ds_in.lat_rho.values.max()}")
print(f"Lon min: {ds_in.lon_rho.values.min()}")
print(f"Lon max: {ds_in.lon_rho.values.max()}")

In [ ]:
lat = np.linspace(59.0, 59.93, num=232)
lon = np.linspace(10.2, 11.02, num=105)

In [ ]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], lat, {"units": "degrees_north"}),
        "lon": (["lon"], lon, {"units": "degrees_east"}),
    }
)

In [ ]:
regridder = xe.Regridder(ds_in.rename({"lon_rho": "lon", "lat_rho": "lat"}), ds_out, "bilinear", unmapped_to_nan=True)

In [ ]:
da = regridder(ds_in["h"])

In [ ]:
da_h = ds_in["h"]

In [ ]:
da_h.plot()

In [ ]:
da = da.where(da > 5, np.nan)
np_depth = da.values

In [ ]:
da.plot(figsize=(7, 10))

In [ ]:
np_depth_new = replace_surrounded_and_clusters(np_depth)

In [ ]:
da.values = np_depth_new

In [ ]:
da.plot(figsize=(7, 10))

In [ ]:
da

In [ ]:
z_faces = np.array([
    -300.0,
    -260.0,
    -200.0,
    -150.0,
    -100.0,
    -75.0,
    -50.0,
    -25.0,
    -15.0,
    -10.0,
    -7.5,
    -5.0,
    -3.0,
    -2.0,
    -1.0,
    0.0,
])

In [ ]:
z_faces_da = xr.DataArray(
    z_faces,
    dims=("z_faces",),
    coords={"z_faces": z_faces},
    name="z_faces",
)

In [ ]:
dsout = xr.Dataset(
    {
        "h": -1 * da.fillna(0),
        "z_faces": z_faces_da,
    }
)

In [ ]:
dsout

In [ ]:
dsout.to_netcdf("/home/sia/FjordsSim_data/oslofjord/OF_inner_105to232_bathymetry.nc")